In [ ]:
import sys, os, io
import pyodbc, pandas
import datetime, tqdm

In [ ]:
def writeInChunks(path, sql, cnxn, chunksize = 10000):
    """
    Writes an sql in chunk to avoid memory problems
    """ 
    with io.open(path, "w", encoding="utf-8") as fh:
        chunks = pandas.read_sql_query(sql, cnxn, chunksize=chunksize)
        next(chunks).to_csv(fh, index=False)  # write the first chunk with the column names,
                                              # but ignore the index (which will be screwed up anyway due to the chunking)
       
        for chunk in chunks:
            print('Chunk written')
            chunk.to_csv(fh, index=False, header=False) # skip the column names from now on


In [ ]:
cnxn = pyodbc.connect(driver='{SQL Server}',
                      server= '192.168.27.210\CMRC',
                      database='SIOS1',
                      uid='CI',
                      pwd='123')
cur = cnxn.cursor()

In [ ]:
sql = "SELECT * FROM Casos WHERE Paciente IN ( SELECT Id FROM Pacientes WHERE Identificacion LIKE 'VEN%')"


In [ ]:
sqlVenezolanas = "SELECT Id FROM Pacientes WHERE Identificacion LIKE 'VEN%'"
def downloadPatientsAndRegistersFromPatients(cnxn, sqlPatients, pathWriteFolder):
    """
    Downloads  all the cases, registers and operations between two dates, plus/minus a certain slack to ensure all registers are fully recovered
    """
    try:
        os.mkdir(pathWriteFolder)
    except:
        pass
    sqlRegistros ="""SELECT * FROM RegistrosHistoria where NumeroHistoria in (%s)  """ % (sqlPatients)
    sqlCasos ="""SELECT * FROM Casos where Paciente in (%s) """ % (sqlPatients)
    sqlProcedimientos ="""SELECT * FROM ProgramacionQx.DescripcionQuirurgica where IdPaciente  in (%s)""" % (sqlPatients)
    sqlPacientes = """SELECT * FROM PACIENTES WHERE Id in (%s)""" %  sqlPatients
    
    sqlCasosID ="""SELECT Caso FROM Casos where Paciente in (%s) """ % (sqlPatients)
    sqlDatosEnfermeria = """SELECT * FROM Enfermeria.SignosVitalesControl WHERE IdAdmision IN (%s)""" % (sqlCasosID)

    
    writeInChunks(os.path.join(pathWriteFolder,'registros.csv'), sqlRegistros, cnxn)
    writeInChunks(os.path.join(pathWriteFolder,'casos.csv'), sqlCasos, cnxn)
    writeInChunks(os.path.join(pathWriteFolder,'procedimientos.csv'), sqlProcedimientos, cnxn)
    writeInChunks(os.path.join(pathWriteFolder,'pacientes.csv'), sqlPacientes, cnxn)
    writeInChunks(os.path.join(pathWriteFolder,'enfermeriaMedidas.csv'), sqlDatosEnfermeria, cnxn)
downloadPatientsAndRegistersFromPatients(cnxn, sqlVenezolanas, 'Venezolanas2')

In [ ]:
def downloadPatientsAndRegistersFromDate(cnxn, startDate, endDate, pathWriteFolder,  slackDays = 15):
    """
    Downloads  all the cases, registers and operations between two dates, plus/minus a certain slack to ensure all registers are fully recovered
    """
    try:
        os.mkdir(pathWriteFolder)
    except:
        pass
    slackDaysDT = datetime.timedelta(days = slackDays)
    startDateDT = datetime.datetime.strptime(startDate, '%d/%m/%Y') + slackDaysDT
    endDateDT = datetime.datetime.strptime(endDate, '%d/%m/%Y') + slackDaysDT
    
    startDateStr =  startDateDT.strftime("%Y-%d-%m")
    endDateStr =endDateDT.strftime("%Y-%d-%m")
    sqlRegistros ="""SELECT * FROM RegistrosHistoria where FechaAsignacionRegistro < '%s' AND FechaAsignacionRegistro > '%s' """ % (endDateStr, startDateStr)
    sqlCasos ="""SELECT * FROM Casos where FechaHora < '%s' AND FechaHora > '%s' """ % (endDateStr, startDateStr)
    sqlProcedimientos ="""SELECT * FROM ProgramacionQx.DescripcionQuirurgica where FechaDescripcion < '%s' AND FechaDescripcion > '%s' """ % (endDateStr, startDateStr)
    sqlProcedimientosIDs = """SELECT DISTINCT IdPaciente FROM ProgramacionQx.DescripcionQuirurgica where FechaDescripcion < '%s' AND FechaDescripcion > '%s' """ % (endDateStr, startDateStr)
    sqlPacientes = """SELECT * FROM PACIENTES WHERE Id in (%s)""" %  sqlProcedimientosIDs
    
    sqlCasosID ="""SELECT Caso FROM Casos where FechaHora < '%s' AND FechaHora > '%s' """ % (endDateStr, startDateStr)
    sqlDatosEnfermeria = """SELECT * FROM Enfermeria.SignosVitalesControl WHERE IdAdmision IN (%s)""" % (sqlCasosID)

    
    writeInChunks(os.path.join(pathWriteFolder,'registros.csv'), sqlRegistros, cnxn)
    writeInChunks(os.path.join(pathWriteFolder,'casos.csv'), sqlCasos, cnxn)
    writeInChunks(os.path.join(pathWriteFolder,'procedimientos.csv'), sqlProcedimientos, cnxn)
    writeInChunks(os.path.join(pathWriteFolder,'pacientes.csv'), sqlPacientes, cnxn)
    writeInChunks(os.path.join(pathWriteFolder,'enfermeriaMedidas.csv'), sqlDatosEnfermeria, cnxn)


In [ ]:
downloadPatientsAndRegistersFromDate(cnxn, '01/01/2019', '01/01/2020', 'casos2019')

In [ ]:
cnxn = pyodbc.connect('Driver={Microsoft Access Driver (*.mdb, *.accdb)};'+ \
                      'DBQ=C:\\sip\\SIP_Respaldo\\sip20200302.mdb;')
cursor = cnxn.cursor()

In [ ]:
df = pandas.read_csv('resultsSIP.csv', index_col = 0)
df['ID01'] = df['VAR_0019'].map(lambda s: '000000000008060061-80000000000' + s)
df['VAR_0198'] = df['VAR_0198'].map(lambda s: str(int(s)) if s == s else s)

In [ ]:
fullCols = list(map(lambda s: s, cursor.columns(table='nivel_N1')))
colNames = list(map(lambda s: s[3], cursor.columns(table='nivel_N1')))
cols = []
values = []
i = 0
for i in tqdm.tqdm_notebook(range(len(df))):
    for c in colNames:

        if c in df and df.iloc[i][c] == df.iloc[i][c]:
            cols.append(c)
            values.append(df.iloc[i][c])
    try:
        cursor.execute("insert into nivel_N1(%s) values (\'%s\')" % ( ','.join(cols), '\',\''.join(map(str,values))))
    except:
        pass
    if i == 1000:
        break
cnxn.commit()

In [ ]:
pandas.read_sql_query("select TOP 5 * from Nivel_N1 where VAR_0019 = 'VEN25322789' ", cnxn)

In [ ]:
df.iloc[0].VAR_0019

In [ ]:
VEN25322789